# Preliminary Data and Functions

In [1]:
import numpy as np
import re
from random import randint

#Importing Data
wordsList = np.load('Numpy Arrays/wordsList.npy')
wordsList = wordsList.tolist()
wordsList = [word.decode('UTF-8') for word in wordsList]
wordVectors = np.load('Numpy Arrays/wordVectors.npy')
#text = np.load('Numpy Arrays/text.npy')
the_labels = np.load('Numpy Arrays/labels.npy')
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

#Helper Functions
def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())
ids = np.load('Numpy Arrays/idsMatrix.npy')
def getTrainBatch():
    label = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1, 1000000)
        arr[i] = ids[num]
        label.append(the_labels[num])
    return arr, label
def getTestBatch():
    label = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1000001, the_labels.shape[0]-1)
        arr[i] = ids[num]
        label.append(the_labels[num])
    return arr, label

# Dense Model

### Hyper Parameters

In [2]:
batchSize = 512
numClasses = 2
iterations = 10000
learning_rate = 0.005
maxSeqLength = 20

In [3]:
import tensorflow as tf
with tf.name_scope("Input_Data") as scope:
    labels = tf.placeholder(tf.float32, [batchSize, numClasses], name='Sentiment_Labels')
    input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength], name='Text')

In [4]:
with tf.name_scope("Data_Preprocessing") as scope:
    data = tf.Variable(tf.zeros([batchSize, maxSeqLength, 3]),dtype=tf.float32, name='Embed_Text')
    data = tf.nn.embedding_lookup(wordVectors,input_data)

In [5]:
with tf.name_scope("Dense_Layer1") as scope:
    nn = tf.layers.dense(inputs=data, activation=tf.nn.relu, use_bias=True, units=32, kernel_initializer=tf.contrib.layers.xavier_initializer())
    nn = tf.nn.dropout(nn, keep_prob=0.9)

In [6]:
with tf.name_scope("Dense_Layer2") as scope:
    nn = tf.layers.dense(inputs=nn, activation=tf.nn.relu, use_bias=True, units=4, kernel_initializer=tf.contrib.layers.xavier_initializer())
    nn = tf.nn.dropout(nn, keep_prob=0.9)

In [7]:
with tf.name_scope("Prediction_Parameters") as scope:
    weight = tf.Variable(tf.truncated_normal([4, numClasses]), name='Weights')
    bias = tf.Variable(tf.constant(0.1, shape=[numClasses]), name='bias')
    value = tf.transpose(nn, [1, 0, 2])
    last = tf.gather(value, int(value.get_shape()[0]) - 1)
with tf.name_scope("Prediction") as scope:
    prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

In [8]:
with tf.name_scope("Accuracy") as scope:
    correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))


In [9]:
with tf.name_scope("Loss") as scope:
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=labels))
with tf.name_scope("Train") as scope:
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [10]:
import datetime
tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
validation = tf.summary.scalar('Validation', accuracy)
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

NameError: name 'sess' is not defined

In [11]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())


In [12]:
for i in range(iterations+1):
        #Next Batch of reviews
    nextBatch, nextBatchLabels = getTrainBatch();
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
    #Write summary to Tensorboard
#    if (i % 5 == 0):
#        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
#        writer.add_summary(summary, i)
#        nextTestBatch, nextTestBatchLabels = getTestBatch();
#        val = sess.run(validation, {input_data: nextTestBatch, labels: nextTestBatchLabels})
#        writer.add_summary(val, i)
    if (i%500 == 0):
        print("Accuracy: " + str(sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})))
save_path = saver.save(sess, "FC2/FC.ckpt", global_step=i)
writer.close()

Accuracy: 0.5253906
Accuracy: 0.55078125
Accuracy: 0.55859375
Accuracy: 0.5527344
Accuracy: 0.5332031
Accuracy: 0.5683594
Accuracy: 0.54296875
Accuracy: 0.578125
Accuracy: 0.5175781
Accuracy: 0.546875
Accuracy: 0.5234375
Accuracy: 0.55078125
Accuracy: 0.51171875
Accuracy: 0.53125
Accuracy: 0.5566406
Accuracy: 0.5605469
Accuracy: 0.5546875
Accuracy: 0.6074219
Accuracy: 0.5917969
Accuracy: 0.52734375
Accuracy: 0.5644531


NameError: name 'writer' is not defined

In [14]:
tf.train.Saver().restore(sess, tf.train.latest_checkpoint("FC2/"))

INFO:tensorflow:Restoring parameters from FC2/FC.ckpt-10000


In [15]:
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix
def test(sent):
    x = ['negetive', 'positive']
    sent = getSentenceMatrix(sent)
    pred = sess.run(prediction, {input_data:sent})[0]
    return(pred)
def results():
    arr = np.array([[1, 0]], dtype=np.float32)
    arr_pred = np.array([test("my life sucks")])
    for i in range(512, ids.shape[0], 512):
        arr = np.vstack((arr, the_labels[i-512:i]))
        arr_pred = np.vstack((arr_pred, sess.run(prediction, {input_data:ids[i-512:i]})))
    x = np.argmax(arr_pred, axis=1)
    y = np.argmax(arr, axis=1)
    return((np.sum(np.equal(x, y))/x.shape))

In [16]:
results()

array([0.5438828])